In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
model_name = "Llama-3.2-3B-Instruct-FineTome20K"
LORA_DIR = f"/content/drive/MyDrive/models/lora/{model_name}"
BASE_MODEL_ID = "unsloth/Llama-3.2-3B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from datasets import load_dataset

# Load FineTome-100k (last 100 samples, not used during finetuning)
fine_ds = load_dataset("mlabonne/FineTome-100k", split="train[20000:20100]")

def extract_prompt_answer(example):
    """
    Extract:
    - first 'human' turn as the user prompt
    - first 'gpt' turn *after* that human turn as the reference answer
    """
    user = None
    assistant = None

    for turn in example["conversations"]:
        if turn["from"] == "human" and user is None:
            user = turn["value"]
        elif turn["from"] == "gpt" and user is not None:
            assistant = turn["value"]
            break

    return {"prompt": user, "reference": assistant}

# Apply extraction
mapped = fine_ds.map(extract_prompt_answer)

# Filter out any weird samples lacking a prompt or reference
eval_subset = mapped.filter(lambda e: e["prompt"] is not None and e["reference"] is not None)

# Drop all columns except prompts and reference that will be used during evaluation
eval_subset = mapped.remove_columns(
    [col for col in mapped.column_names if col not in ["prompt", "reference"]]
)

print("Eval subset size:", len(eval_subset))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Eval subset size: 100


In [5]:
eval_subset[1]

{'prompt': 'Write Python code to solve the task:\nIn 1976 the “Four Color Map Theorem” was proven with the assistance of a computer. This theorem\nstates that every map can be colored using only four colors, in such a way that no region is colored\nusing the same color as a neighbor region.\n\nHere you are asked to solve a simpler similar problem. You have to decide whether a given arbitrary\nconnected graph can be bicolored. That is, if one can assign colors (from a palette of two) to the nodes\nin such a way that no two adjacent nodes have the same color. To simplify the problem you can assume:\n\n• no node will have an edge to itself.\n\n• the graph is nondirected. That is, if node a is said to be connected to node b, then you must\nassume that b is connected to a.\n\n• the graph will be strongly connected. That is, there will be at least one path from any node to\nany other node.\n\nInput\n\nFirst line of input contains the total number of test cases T.\nEach test case starts with 

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [7]:
from peft import PeftModel
from tqdm import tqdm

def generate_all_answers(model, tokenizer, eval_subset, max_new_tokens=256):
    answers = []
    for ex in tqdm(eval_subset, desc="Answers"):
        prompt = ex["prompt"]
        messages = [{"role": "user", "content": prompt}]
        prompt_str = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        inputs = tokenizer(
            prompt_str,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
        ).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
            )

        gen_ids = outputs[0][inputs["input_ids"].shape[1]:]
        answer = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
        answers.append(answer)
    return answers

In [8]:
"""Original base model"""
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
  BASE_MODEL_ID,
  torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
  device_map="auto" if DEVICE == "cuda" else None,
)
model.eval()
print("Loaded base model")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded base model


In [9]:
base_answers = generate_all_answers(model, tokenizer, eval_subset)

Answers: 100%|██████████| 100/100 [13:53<00:00,  8.34s/it]


In [10]:
# Reset GPU VRAM
model.to("cpu")
del model
torch.cuda.empty_cache()

In [11]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
    device_map="auto" if DEVICE == "cuda" else None,
)
ft_model = PeftModel.from_pretrained(base_model, LORA_DIR)
ft_model.eval()
print("Loaded finetuned model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded finetuned model


In [12]:
ft_answers = generate_all_answers(ft_model, tokenizer, eval_subset)

Answers: 100%|██████████| 100/100 [28:34<00:00, 17.15s/it]


In [13]:
def pretty_print_comparison(eval_subset, base_answers, ft_answers, n=None):
    """
    Print prompt, reference, and both model answers in a clean format.
    n = how many samples to print (default = all)
    """
    if n is None:
        n = len(eval_subset)

    for i in range(n):
        ex = eval_subset[i]
        base = base_answers[i]
        ft = ft_answers[i]

        print("=" * 80)
        print(f"🧩 Sample {i}")
        print("=" * 80)

        print("\n🎯 PROMPT:")
        print(ex["prompt"])

        print("\n📘 REFERENCE ANSWER:")
        print(ex["reference"])

        print("\n🤖 BASE MODEL ANSWER:")
        print(base)

        print("\n🚀 FINETUNED MODEL ANSWER:")
        print(ft)

        print("\n" + "-" * 80 + "\n")


Note that there is some problems with the reference in some cases. Here is an example where both the default and finetuned model answers better than the reference. Therefore we exclude the reference in the LLM as a judge comparison later.

In [14]:
pretty_print_comparison(eval_subset, base_answers, ft_answers, n=1)


🧩 Sample 0

🎯 PROMPT:
Write Python code to solve the task:
Connell Sequence is the sequence formed with the first odd number, i.e 1 as its first term. The subsequent terms of the sequence are made up of the first two even numbers, i.e 2 and 4, followed by the next three odd numbers, i.e 5, 7 and 9, followed by the next four even numbers, i.e 10, 12, 14 and 16 and so on …. 
Given an integer n, generate the first n terms of the Connell Sequence.
Example 1:
Input: n = 6
Output: 1 2 4 5 7 9
Explaination: First one odd number. Then 
two even numbers. Then three odd numbers.
 
Example 2:
Input: n = 2
Output: 1 2
Explaination: The sequence consists of first 
odd number then one even number.
 
Your Task:
You do not need to read input or print anything. Your task is to complete the function connellSequence() which takes n as input parmeters and returns a list of integers containing the first n numbers of the Connell sequence.
 
Expected Time Complexity: O(n)
Expected Auxiliary Space: O(n)
 
Con

In [15]:
# model.to("cpu")
# del model
# torch.cuda.empty_cache()

In [16]:
!pip install --upgrade openai


In [17]:
from openai import OpenAI
client = OpenAI()  # uses OPENAI_API_KEY env var


In [18]:
JUDGE_SYSTEM_PROMPT = """You are an impartial evaluator comparing two AI assistant answers.

You will be given:
- A user instruction.
- Candidate answer A.
- Candidate answer B.

You must decide which answer is better overall for the user, considering:
- Correctness and factual accuracy
- Following the instruction
- Clarity and coherence
- Completeness and level of detail
- Safety (no harmful or disallowed content)

If one answer is clearly better, choose that one.
If they are roughly equally good (or both equally bad), choose "tie".

Reply with exactly ONE TOKEN among:
A
B
tie
No explanation, just "A", "B", or "tie".
"""


In [19]:
def judge_pair_openai(prompt, base_answer, ft_answer, model_name="gpt-4.1-mini"):
    """
    Returns: "A", "B", or "tie"
    A = base_answer is better
    B = ft_answer is better
    """
    user_prompt = f"""User instruction:
{prompt}

Candidate A:
{base_answer}

Candidate B:
{ft_answer}

Between Candidate A and Candidate B, which answer is better overall for the user?
Reply with exactly one word: A, B, or tie.
"""

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=1,
        temperature=0.0,
    )
    choice = response.choices[0].message.content.strip()
    print(choice)
    # Safety: normalize a bit
    choice = choice.upper()
    if choice not in ["A", "B", "TIE"]:
        # fallback if judge misbehaves
        choice = "TIE"
    return choice


In [27]:
from collections import Counter
from tqdm import tqdm

def evaluate_with_judge(eval_subset, base_answers, ft_answers, model_name="gpt-4.1-mini", max_samples=None):
    """
    Run LLM-as-a-judge pairwise comparison.
    Also store all non-tie samples for later inspection.
    """

    assert len(eval_subset) == len(base_answers) == len(ft_answers), "Length mismatch!"

    if max_samples is None:
        max_samples = len(eval_subset)

    decisions = []
    detailed = [] # store all samples + decision
    non_ties = [] # store only A/B

    for i in tqdm(range(max_samples), desc="Judging"):
        prompt = eval_subset[i]["prompt"]
        base = base_answers[i]
        ft = ft_answers[i]

        decision = judge_pair_openai(prompt, base, ft, model_name=model_name)
        decisions.append(decision)

        record = {
            "idx": i,
            "prompt": prompt,
            "base_answer": base,
            "ft_answer": ft,
            "decision": decision,
        }
        detailed.append(record)

        if decision in ["A", "B"]:
            non_ties.append(record)

    # summary
    counts = Counter(decisions)
    total = len(decisions)

    print("=== Judge results ===")
    print(f"Total comparisons: {total}")
    print(f"Base wins (A):     {counts.get('A', 0)}  ({counts.get('A', 0)/total:.1%})")
    print(f"FT wins   (B):     {counts.get('B', 0)}  ({counts.get('B', 0)/total:.1%})")
    print(f"Ties:             {counts.get('TIE', 0)} ({counts.get('TIE', 0)/total:.1%})")

    return decisions, counts, detailed, non_ties


In [21]:
decisions, counts, detailed, non_ties = evaluate_with_judge(
    eval_subset=eval_subset,
    base_answers=base_answers,
    ft_answers=ft_answers,
    model_name="gpt-4.1-mini",
    max_samples=100,
)

Judging:   1%|          | 1/100 [00:00<01:35,  1.03it/s]

A


Judging:   2%|▏         | 2/100 [00:01<01:08,  1.44it/s]

A


Judging:   3%|▎         | 3/100 [00:02<01:08,  1.41it/s]

...

B


Judging:  99%|█████████▉| 99/100 [00:56<00:00,  1.60it/s]

tie


Judging: 100%|██████████| 100/100 [00:56<00:00,  1.76it/s]

tie
=== Judge results ===
Total comparisons: 100
Base wins (A):     42  (42.0%)
FT wins   (B):     31  (31.0%)
Ties:             27 (27.0%)


This cell runs the eval providing the answers of the model in the other order.

In [28]:
def pretty_print_comparison_of_ties(record):
    print("="*80)
    print(f"Sample index: {record['idx']}  (decision={record['decision']})")
    print("="*80)

    print("\n🎯 PROMPT:")
    print(record["prompt"])

    print("\n🤖 BASE ANSWER:")
    print(record["base_answer"])

    print("\n🚀 FT ANSWER:")
    print(record["ft_answer"])

    print("\n" + "-"*80 + "\n")

In [29]:
def evaluate_with_judge_opposite(eval_subset, base_answers, ft_answers, model_name="gpt-4.1-mini", max_samples=None):
    """
    Run LLM-as-a-judge pairwise comparison.
    Also store all non-tie samples for later inspection.
    """

    assert len(eval_subset) == len(base_answers) == len(ft_answers), "Length mismatch!"

    if max_samples is None:
        max_samples = len(eval_subset)

    decisions = []
    detailed = [] # store all samples + decision
    non_ties = [] # store only A/B

    for i in tqdm(range(max_samples), desc="Judging"):
        prompt = eval_subset[i]["prompt"]
        base = base_answers[i]
        ft = ft_answers[i]

        decision = judge_pair_openai(prompt, ft, base, model_name=model_name)
        decisions.append(decision)

        record = {
            "idx": i,
            "prompt": prompt,
            "base_answer": base,
            "ft_answer": ft,
            "decision": decision,
        }
        detailed.append(record)

        if decision in ["A", "B"]:
            non_ties.append(record)

    # summary
    counts = Counter(decisions)
    total = len(decisions)

    print("=== Judge results ===")
    print(f"Total comparisons: {total}")
    print(f"FT wins (A):     {counts.get('A', 0)}  ({counts.get('A', 0)/total:.1%})")
    print(f"Base wins   (B):     {counts.get('B', 0)}  ({counts.get('B', 0)/total:.1%})")
    print(f"Ties:             {counts.get('TIE', 0)} ({counts.get('TIE', 0)/total:.1%})")

    return decisions, counts, detailed, non_ties

In [33]:
decisions, counts, detailed, non_ties = evaluate_with_judge_opposite(
    eval_subset=eval_subset,
    base_answers=base_answers,
    ft_answers=ft_answers,
    model_name="gpt-4.1-mini",
    max_samples=100,
)

Judging:   1%|          | 1/100 [00:00<00:47,  2.08it/s]

B


Judging:   2%|▏         | 2/100 [00:00<00:47,  2.07it/s]

B


Judging:   3%|▎         | 3/100 [00:01<00:49,  1.98it/s]

...

tie


Judging:  99%|█████████▉| 99/100 [00:53<00:00,  1.60it/s]

tie


Judging: 100%|██████████| 100/100 [00:54<00:00,  1.84it/s]

tie
=== Judge results ===
Total comparisons: 100
FT wins (A):     32  (32.0%)
Base wins   (B):     39  (39.0%)
Ties:             29 (29.0%)


See how the judge does if prompts are exactly the same

In [31]:
decisions, counts, detailed, non_ties = evaluate_with_judge(
    eval_subset=eval_subset,
    base_answers=ft_answers,
    ft_answers=ft_answers,
    model_name="gpt-4.1-mini",
    max_samples=10,
)

Judging:  10%|█         | 1/10 [00:00<00:04,  2.04it/s]

tie


Judging:  20%|██        | 2/10 [00:00<00:03,  2.15it/s]

tie


Judging:  30%|███       | 3/10 [00:01<00:03,  2.13it/s]

...

tie


Judging:  90%|█████████ | 9/10 [00:04<00:00,  1.88it/s]

tie


Judging: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]

tie
=== Judge results ===
Total comparisons: 10
Base wins (A):     0  (0.0%)
FT wins   (B):     0  (0.0%)
Ties:             10 (100.0%)


Print all the non-ties to double check that the LLM is correct.

In [34]:
for rec in non_ties:
    pretty_print_comparison_of_ties(rec)


Sample index: 0  (decision=B)

🎯 PROMPT:
Write Python code to solve the task:
Connell Sequence is the sequence formed with the first odd number, i.e 1 as its first term. The subsequent terms of the sequence are made up of the first two even numbers, i.e 2 and 4, followed by the next three odd numbers, i.e 5, 7 and 9, followed by the next four even numbers, i.e 10, 12, 14 and 16 and so on …. 
Given an integer n, generate the first n terms of the Connell Sequence.
Example 1:
Input: n = 6
Output: 1 2 4 5 7 9
Explaination: First one odd number. Then 
two even numbers. Then three odd numbers.
 
Example 2:
Input: n = 2
Output: 1 2
Explaination: The sequence consists of first 
odd number then one even number.
 
Your Task:
You do not need to read input or print anything. Your task is to complete the function connellSequence() which takes n as input parmeters and returns a list of integers containing the first n numbers of the Connell sequence.
 
Expected Time Complexity: O(n)
Expected Auxiliar